## 캐글 데이터셋

In [11]:
## 파일 경로
## 문장
# file_path_train_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/train.hate.csv"
# file_path_dev_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/dev.hate.csv"
# file_path_unlabeled_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/unlabeled_comments.txt"
# file_path_no_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/test.hate.no_label.csv"
# file_path_kold_v1 = "/content/drive/MyDrive/공부/자연어 처리/자료/kold_v1.json"
# file_path_youtube_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/comments_update_df.csv"
# file_path_trip = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/tripadviser_review.csv"
# file_path_drama = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/Dataset.csv"
# file_path_common = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/common_word.csv"

# ## 단어
# file_path_pos_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/pos_word.csv"
# file_path_neg_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/neg_word.csv"
# file_path_unknown = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/obj_unknown_pol_word.txt"
# file_path_senti_dict = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/SentiWord_Dict.txt"
# file_path_stopwords = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/stopwords.txt"

file_path_change_label = './데이터/changed_label.csv'
file_path_stopwords = './데이터/불용어사전.txt'

In [12]:
"""
[문장]
df_train_hate : 
 - label 있음.
 - none         3486
 - offensive    2499
 - hate         1911


df_dev_hate :
 - label 있음.
 - offensive    189
 - none         160
 - hate         122


df_unlabeled_comments : label 없음
df_no_hate : label 없음
df_kold_v1 : label 없음
df_youtube_comments : label 없음,  좋아요 수 있음


[단어]
pos_word.csv
neg_word.csv
obj_unknown_pol_word.txt
SentiWord_Dict.txt
"""

import pandas as pd
import numpy as np


## 문장
# df_train_hate = pd.read_csv(file_path_train_hate, encoding='utf-8', engine='python')
# df_dev_hate = pd.read_csv(file_path_dev_hate, encoding='utf-8', engine='python')
# df_unlabeled_comments = pd.read_table(file_path_unlabeled_comments, encoding='utf-8', engine='python', names=['comment'],)
# df_no_hate = pd.read_csv(file_path_no_hate, encoding='utf-8', engine='python')
# df_kold_v1 = pd.read_json(file_path_kold_v1, encoding='utf-8')
# df_youtube_comments = pd.read_csv(file_path_youtube_comments, encoding='utf-8', engine='python')
# df_trip = pd.read_csv(file_path_trip, encoding='utf-8', engine='python')
# df_kdrama_common = pd.read_csv(file_path_common, encoding='utf-8', engine='python',)


## 단어
# df_pos_word = pd.read_csv(file_path_pos_word, encoding='utf-8', engine='python')
# df_neg_word = pd.read_csv(file_path_neg_word, encoding='utf-8', engine='python')
# df_unknown_word = pd.read_table(file_path_unknown, encoding='utf-8', engine='python', names=['word'])
new_df = pd.read_csv(file_path_change_label, encoding='utf-8', engine='python')
new_df

,text,y
0,여행에 집중할수 있게 편안한 휴식을 제공하는 호텔이었습니다. 위치선정 또한 적당한 ...,3
1,"2일 이상 연박시 침대, 이불, 베게등 침구류 교체 및 어메니티 보강이 필요해 보입...",3
2,지인에소개로온 호텔 깨끗하고 좋은거같아요 처음에는 없는게 많아 많이 당황했는데 ...,3
3,방에 딱 들어서자마자 눈이 휘둥그레질정도로 이렇게 넓은 호텔 처음 와본 것 같아요!...,3
4,저녁에 맥주한잔 하는게 좋아서 렌트 안하고 뚜벅이 하기로 했는데 호텔 바로 앞에 버...,3
...,...,...
16159,안정감,3
16160,적극적이다,3
16161,승리,3
16162,승리하다,3


In [35]:
new_df.y.value_counts()

3    5572
1    5420
2    5172
Name: y, dtype: int64

In [13]:
new_df.y.value_counts()

3    5572
1    5420
2    5172
Name: y, dtype: int64

In [14]:
new_df.isnull().sum()

text    0
y       0
dtype: int64

## 자연어 처리 시작

### 자연어 처리 모듈 임포트

In [15]:
import Preprocessing as prep
import Modeling as mo

In [16]:
stopwords_list = ['제주도', '호스텔', '위치', '대중교통', '타고', '어디', '공항', '잠깐', '직원', '택시', '상품',]
stopwords_list = prep.stopwords(file_path_stopwords, stopwords_list)

In [17]:
prep.text_cleaning(new_df.text[5400], stopwords_list)

['어쩌면', '이다해', '승리', '승리']

### 문서-단어 행렬 생성

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer= lambda x: prep.text_cleaning(x, stopwords_list))
bow_vect = vect.fit_transform(new_df['text'].tolist())

In [19]:
word_list = vect.get_feature_names_out()
count_list = bow_vect.toarray().sum(axis=0)
count_list

array([ 5,  1, 15, ...,  1,  1,  1], dtype=int64)

In [39]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict

{'가가': 5,
 '가가린': 1,
 '가게': 15,
 '가격': 262,
 '가격표': 1,
 '가관': 2,
 '가구': 12,
 '가급': 1,
 '가기': 23,
 '가까이': 24,
 '가끔': 10,
 '가나': 4,
 '가난': 5,
 '가난뱅이': 1,
 '가년': 1,
 '가능': 12,
 '가능성': 4,
 '가다가': 1,
 '가담': 1,
 '가당': 1,
 '가도': 8,
 '가동': 2,
 '가두': 1,
 '가드': 1,
 '가득': 5,
 '가든': 2,
 '가라': 6,
 '가락': 2,
 '가래': 2,
 '가량': 5,
 '가려움': 1,
 '가렴': 1,
 '가렵': 1,
 '가로': 7,
 '가로본능폰': 1,
 '가로세로': 3,
 '가로수길': 1,
 '가루': 2,
 '가르침': 3,
 '가마니': 1,
 '가면': 21,
 '가몬': 1,
 '가무': 1,
 '가문': 1,
 '가물': 1,
 '가뭄': 2,
 '가미': 1,
 '가방': 4,
 '가버': 1,
 '가보': 1,
 '가봉': 1,
 '가쁘': 8,
 '가쁜': 2,
 '가사': 5,
 '가성': 51,
 '가세': 9,
 '가소': 1,
 '가수': 29,
 '가스': 1,
 '가스레인지': 1,
 '가스렌지': 1,
 '가슴': 22,
 '가슴앓이': 2,
 '가시': 4,
 '가식': 2,
 '가신': 4,
 '가안': 1,
 '가야': 20,
 '가오': 1,
 '가옥': 1,
 '가요': 17,
 '가요무대': 1,
 '가요제': 1,
 '가용': 2,
 '가운데': 4,
 '가운데손가락': 1,
 '가위': 1,
 '가을': 5,
 '가인': 3,
 '가장': 55,
 '가점': 1,
 '가정': 14,
 '가정불화': 1,
 '가정사': 2,
 '가정식': 2,
 '가정폭력': 3,
 '가족': 137,
 '가족사': 1,
 '가족사진': 1,
 '가족영화': 2,
 '가즈': 11,
 '가지': 77,
 '가짐': 3,
 '가짓수':

### TF-IDF 생성

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
tfidf_vect = tfidf.fit_transform(bow_vect)
tfidf_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 단어 맵핑

In [41]:
invert_index_vectorizer = {v : k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer[2928]

'무궁무진'

In [42]:
vect.vocabulary_

{'여행': 5870,
 '집중': 8197,
 '휴식': 10086,
 '제공': 7680,
 '호텔': 9883,
 '선정': 4441,
 '청소': 8419,
 '청결': 8413,
 '상태': 4285,
 '침대': 8673,
 '이불': 6794,
 '침구': 8672,
 '교체': 821,
 '어메니티': 5652,
 '보강': 3608,
 '베스트': 3539,
 '웨스턴': 6447,
 '회원': 10025,
 '경우': 487,
 '객실': 299,
 '층수': 8635,
 '요청': 6274,
 '적극': 7441,
 '반영': 3288,
 '지인': 8087,
 '소개': 4594,
 '처음': 8367,
 '당황': 1838,
 '세면': 4546,
 '도구': 1994,
 '잠옷': 7290,
 '필수': 9439,
 '그것': 1000,
 '정도': 7594,
 '박만': 3221,
 '저녁': 7422,
 '맥주': 2691,
 '한잔': 9578,
 '렌트': 2382,
 '뚜벅': 2290,
 '버스': 3468,
 '정류': 7599,
 '도착': 2046,
 '가방': 47,
 '일찍': 7087,
 '정비': 7614,
 '방이': 3382,
 '입실': 7136,
 '업그레이드': 5747,
 '정말': 7604,
 '바다': 3183,
 '전망': 7484,
 '전일': 7516,
 '함덕': 9610,
 '대명': 1871,
 '콘도': 8799,
 '실내': 5198,
 '분위기': 3843,
 '손님': 4679,
 '가장': 79,
 '배치': 3428,
 '대해': 1933,
 '대응': 1898,
 '써비스': 5292,
 '조식': 7770,
 '부분': 3763,
 '신경': 5119,
 '아주': 5376,
 '특급': 9086,
 '트랜디': 9059,
 '고민': 557,
 '흔적': 10116,
 '여름': 5838,
 '수영장': 4788,
 '사용': 4142,
 '보고': 3612,
 '엄마':

### 모델링

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'n_estimators' : [100, 300, 500, 700, 1000, 1500, 2000, 3000],
    'max_depth' : [1, 2 ,3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_split' : [1, 2 ,3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf' : [1, 2 ,3, 4, 5, 6, 7, 8, 9, 10],
    'max_features' : [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'bootstrap' : [True, False],
    'n_jobs' : [-1, 0, 1, 2 ,3, 4, 5, 6, 7, 8, 9, 10],
}



x_train, x_test, y_train, y_test = train_test_split(tfidf_vect, new_df['y'], shuffle=True, test_size=0.2, stratify=new_df.y, random_state=33)

grid = GridSearchCV(RandomForestClassifier(), params, cv=3, n_jobs=-1)

# model = RandomForestClassifier(random_state=33)
y_pred = grid.fit(x_train, y_train)
# ?RandomForestClassifier

In [27]:
accuracy_score(y_pred, y_test)
    
    
    
    
    
    
    

0.7200742344571606

In [28]:
None, 'micro', 'macro', 'weighted', 'samples'

(None, 'micro', 'macro', 'weighted', 'samples')

In [29]:
recall_score(y_pred, y_test, average='macro')

0.7435120405543785

In [30]:
precision_score(y_pred, y_test, average='macro')

0.7258267386778264

In [31]:
f1_score(y_pred, y_test, average='macro')

0.7099604003400316

In [32]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           1       0.80      0.62      0.70      1398
           2       0.93      0.78      0.85      1230
           3       0.45      0.83      0.58       605

    accuracy                           0.72      3233
   macro avg       0.73      0.74      0.71      3233
weighted avg       0.78      0.72      0.73      3233



In [33]:
confusion_matrix(y_pred, y_test)

array([[866,  46, 486],
       [145, 959, 126],
       [ 73,  29, 503]], dtype=int64)

## YoutubeAPI 실습

In [3]:
from CrawlingModule import YoutubeBulider

def api_key():
    infile = open("C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/api_key.txt", 'r')
    key = infile.readline()
    return key
    

In [ ]:
# 실시간 인기 동영상
build = YoutubeBulider(api_key())
most_pop_videos = build.get_videos(chart='UCQ2O-iftmnlfrBuNsUUTofQ')
most_pop_videos

In [10]:
import json

with open('most_popular_videos.json', 'w') as json_file:
    json.dump(most_pop_videos, json_file)

In [5]:
from CrawlingModule import YoutubeBulider

bulider = YoutubeBulider(api_key())

# Channel_id ㅇㄷ?


# yotube_API를 이용한 인기 급상승 video_id 가져오기
video_list = bulider.get_videoId_in_channel(channelId='UCZ0bi2aVJngKLwFTU5g_fLQ')
categoryId_list = bulider.get_categoryId_in_channel(videoId_list=video_list)

print(len(categoryId_list))
categoryId_list

100


['20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '25',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20',
 '20']

In [24]:
import pandas as pd 

df = pd.DataFrame(
    {
        'video_id' : video_list, 
        'category_id' : list(map(int, categoryId_list)),
        'category_name' : 'N',
    }
)
# df.category_id = df.category_id.astype(int)infile = open('./category_id_list.txt', 'r').readlines()

infile = open('../자료/category_id_value.txt', 'r').readlines()

category_key_value = {}

for line in infile:
    key_value_list = line.split(':')
    category_key_value[int(key_value_list[0])] = key_value_list[1][1:-2]

def invert_category_name(category_Id:int) -> str:
    return category_key_value[category_Id]

df.category_name = df.category_id.apply(invert_category_name)
df


,video_id,category_id,category_name
0,Ynwm92g2hxQ,20,Gamin
1,a4abVmaMj5A,20,Gamin
2,TUTMzIP_Mv0,20,Gamin
3,zrN-ybe49lg,20,Gamin
4,PuRqyxLxTTc,20,Gamin
...,...,...,...
95,u9zKB3s-R1A,20,Gamin
96,De0mePZH0rI,20,Gamin
97,Bb43QlfkCAE,20,Gamin
98,wEa_8r0R3P8,20,Gamin


In [21]:
df.category_name.value_counts()

Gamin             99
News & Politic     1
Name: category_name, dtype: int64